# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments  \
0  Mar 17 2023  4:27PM  259521        22       643957   
1  Mar 17 2023  4:16PM  259520        22       643950   
2  Mar 17 2023  3:08PM  259519        16    SSF_C1W12   
3  Mar 17 2023  2:40PM  259518        19       N17997   
4  Mar 17 2023  2:30PM  259516        19  ADV80013980   
5  Mar 17 2023  2:30PM  259516        19  ADV80013981   
6  Mar 17 2023  2:30PM  259516        19  ADV80013982   
7  Mar 17 2023  2:30PM  259516        19  ADV80013983   
8  Mar 17 2023  2:30PM  259516        19  ADV80013984   
9  Mar 17 2023  2:30PM  259516        19  ADV80013985   

                        Customer ShipmentStatus  
0              NBTY Global, Inc.       Released  
1              NBTY Global, Inc.       Released  
2  Sartorius Stedim Filters Inc.       Released  
3         NAPP Technologies, LLC       Released  
4            AdvaGen Pharma, Ltd      Executing  
5            AdvaGen Pharma, Ltd      Executing  
6            AdvaGen Pharma, Ltd      Executing  
7            AdvaGen Pharma, Ltd      Executing  
8            AdvaGen Pharma, Ltd      Completed  
9            AdvaGen Pharma, Ltd      Completed

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
7   259516      Executing          6
8   259518       Released          1
9   259519       Released          1
10  259520       Released          1
11  259521       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
259516                2.0        6.0       NaN
259518                NaN        NaN       1.0
259519                NaN        NaN       1.0
259520                NaN        NaN       1.0
259521                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259478               20.0        2.0       0.0
259494                0.0        0.0       1.0
259499                0.0        1.0       0.0
259506                0.0        0.0       1.0
259509                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259478                 20          2         0
259494                  0          0         1
259499                  0          1         0
259506                  0          0         1
259509                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               259478         20          2         0
1               259494          0          0         1
2               259499          0          1         0
3               259506          0          0         1
4               259509          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               259478        20         2         
1               259494                            1
2               259499                   1         
3               259506                            1
4               259509                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                       Customer
0   Mar 17 2023  4:27PM  259521        22              NBTY Global, Inc.
1   Mar 17 2023  4:16PM  259520        22              NBTY Global, Inc.
2   Mar 17 2023  3:08PM  259519        16  Sartorius Stedim Filters Inc.
3   Mar 17 2023  2:40PM  259518        19         NAPP Technologies, LLC
4   Mar 17 2023  2:30PM  259516        19            AdvaGen Pharma, Ltd
12  Mar 17 2023  1:55PM  259509        22              NBTY Global, Inc.
13  Mar 17 2023  1:36PM  259506        19           Emersa Waterbox, LLC
14  Mar 17 2023  1:06PM  259499        18             Innova Softgel LLC
15  Mar 17 2023 12:26PM  259494        22              NBTY Global, Inc.
16  Mar 17 2023  7:49AM  259478        19            AdvaGen Pharma, Ltd

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                       Customer  \
0  Mar 17 2023  4:27PM  259521        22              NBTY Global, Inc.   
1  Mar 17 2023  4:16PM  259520        22              NBTY Global, Inc.   
2  Mar 17 2023  3:08PM  259519        16  Sartorius Stedim Filters Inc.   
3  Mar 17 2023  2:40PM  259518        19         NAPP Technologies, LLC   
4  Mar 17 2023  2:30PM  259516        19            AdvaGen Pharma, Ltd   
5  Mar 17 2023  1:55PM  259509        22              NBTY Global, Inc.   
6  Mar 17 2023  1:36PM  259506        19           Emersa Waterbox, LLC   
7  Mar 17 2023  1:06PM  259499        18             Innova Softgel LLC   
8  Mar 17 2023 12:26PM  259494        22              NBTY Global, Inc.   
9  Mar 17 2023  7:49AM  259478        19            AdvaGen Pharma, Ltd   

  Completed Executing Released  
0                            1  
1                            1  
2                            1  
3                            1  
4         2         6           
5                            1  
6                            1  
7                   1           
8                            1  
9        20         2

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                       Customer  \
0  Mar 17 2023  4:27PM  259521        22              NBTY Global, Inc.   
1  Mar 17 2023  4:16PM  259520        22              NBTY Global, Inc.   
2  Mar 17 2023  3:08PM  259519        16  Sartorius Stedim Filters Inc.   
3  Mar 17 2023  2:40PM  259518        19         NAPP Technologies, LLC   
4  Mar 17 2023  2:30PM  259516        19            AdvaGen Pharma, Ltd   
5  Mar 17 2023  1:55PM  259509        22              NBTY Global, Inc.   
6  Mar 17 2023  1:36PM  259506        19           Emersa Waterbox, LLC   
7  Mar 17 2023  1:06PM  259499        18             Innova Softgel LLC   
8  Mar 17 2023 12:26PM  259494        22              NBTY Global, Inc.   
9  Mar 17 2023  7:49AM  259478        19            AdvaGen Pharma, Ltd   

  Released Executing Completed  
0        1                      
1        1                      
2        1                      
3        1                      
4                  6         2  
5        1                      
6        1                      
7                  1            
8        1                      
9                  2        20

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Mar 17 2023  4:27PM  259521        22              NBTY Global, Inc.   
1  Mar 17 2023  4:16PM  259520        22              NBTY Global, Inc.   
2  Mar 17 2023  3:08PM  259519        16  Sartorius Stedim Filters Inc.   
3  Mar 17 2023  2:40PM  259518        19         NAPP Technologies, LLC   
4  Mar 17 2023  2:30PM  259516        19            AdvaGen Pharma, Ltd   

  Released Executing Completed  
0        1                      
1        1                      
2        1                      
3        1                      
4                  6         2

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Mar 17 2023  4:27PM  259521        22              NBTY Global, Inc.   
1  Mar 17 2023  4:16PM  259520        22              NBTY Global, Inc.   
2  Mar 17 2023  3:08PM  259519        16  Sartorius Stedim Filters Inc.   
3  Mar 17 2023  2:40PM  259518        19         NAPP Technologies, LLC   
4  Mar 17 2023  2:30PM  259516        19            AdvaGen Pharma, Ltd   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       1.0        NaN        NaN  
2       1.0        NaN        NaN  
3       1.0        NaN        NaN  
4       NaN        6.0        2.0

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Mar 17 2023  4:27PM  259521        22              NBTY Global, Inc.   
1  Mar 17 2023  4:16PM  259520        22              NBTY Global, Inc.   
2  Mar 17 2023  3:08PM  259519        16  Sartorius Stedim Filters Inc.   
3  Mar 17 2023  2:40PM  259518        19         NAPP Technologies, LLC   
4  Mar 17 2023  2:30PM  259516        19            AdvaGen Pharma, Ltd   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       1.0        0.0        0.0  
2       1.0        0.0        0.0  
3       1.0        0.0        0.0  
4       0.0        6.0        2.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
16          259519       1.0        0.0        0.0
18          259499       0.0        1.0        0.0
19         1038018       2.0        8.0       22.0
22         1038044       4.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        16   259519       1.0        0.0        0.0
1        18   259499       0.0        1.0        0.0
2        19  1038018       2.0        8.0       22.0
3        22  1038044       4.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        16       1.0        0.0        0.0
1        18       0.0        1.0        0.0
2        19       2.0        8.0       22.0
3        22       4.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        16   Released    1.0
1        18   Released    0.0
2        19   Released    2.0
3        22   Released    4.0
4        16  Executing    0.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse   16   18    19   22
Status                        
Completed  0.0  0.0  22.0  0.0
Executing  0.0  1.0   8.0  0.0
Released   1.0  0.0   2.0  4.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status   16   18    19   22
0          Completed  0.0  0.0  22.0  0.0
1          Executing  0.0  1.0   8.0  0.0
2           Released  1.0  0.0   2.0  4.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status   16   18    19   22
0  Completed  0.0  0.0  22.0  0.0
1  Executing  0.0  1.0   8.0  0.0
2   Released  1.0  0.0   2.0  4.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()